## 第一部分：了解 nn.Module的基本操作

In [8]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

In [9]:
model = models.resnet18()

### 打印出 model底下所有 parameters 的 name 以及對應的 shape 

In [14]:
for name, param in model.named_parameters():
    print(name, param)

conv1.weight Parameter containing:
tensor([[[[-2.2341e-02, -3.3889e-02, -2.8335e-03,  ...,  2.1644e-02,
           -1.5689e-02,  1.1426e-02],
          [-8.8924e-03,  4.9406e-05, -1.2507e-02,  ..., -5.4263e-03,
           -5.2147e-03, -1.1164e-02],
          [ 7.8071e-03, -1.1825e-02, -1.7655e-02,  ..., -4.8215e-02,
            5.4065e-04, -2.8641e-02],
          ...,
          [ 6.0948e-02,  5.5251e-02,  1.0213e-02,  ...,  2.7272e-04,
           -2.2279e-02,  1.5546e-02],
          [ 9.2941e-03, -3.8155e-03,  8.0587e-03,  ...,  1.4657e-02,
            1.7075e-03, -2.5105e-02],
          [ 1.3979e-02, -2.7973e-02,  2.1164e-02,  ..., -1.2521e-02,
           -4.0000e-02,  1.5469e-02]],

         [[-2.9564e-03,  2.7203e-02, -4.0392e-02,  ...,  1.7140e-03,
            1.1884e-02,  4.3149e-02],
          [ 9.2776e-03,  9.8492e-03,  1.9534e-02,  ...,  2.5174e-02,
            1.7396e-02, -1.0401e-02],
          [ 1.4074e-02,  2.9998e-02,  1.8647e-03,  ...,  2.7235e-02,
           -2.1336e-02,

### 為了使 forward propagation 加速 並降低 memory 使用量，請將所有 parameters 的requires_grad 關閉，關閉之後執行  forward propagation

In [15]:
input_ = torch.randn(1, 3, 128, 128)

In [16]:
for param in model.parameters():
    param.requires_grad = False

In [17]:
output = model(input_)
print(output.shape)

torch.Size([1, 1000])


## 第二部分：依照指令，以較簡潔的方式搭建出模型

* input_shape = torch.Size([10, 12])
* 先經過一層 nn.Linear(12, 10)
* 經過10層 nn.Linear(10, 10)
* 最後經過 nn.Linear(10, 3) 輸出
* 每一個 nn.Linear過完後要先經過 nn.BatchNorm1d 才能到下一層，輸出層不用


In [18]:
input_ = torch.randn(10,12)
## 示範
Linear = nn.Linear(12,10)
BN = nn.BatchNorm1d(10)

In [19]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.sequential = nn.Sequential(nn.Linear(12,10), nn.BatchNorm1d(10))
        self.repeat_linear = nn.ModuleList([nn.Sequential(nn.Linear(10,10), nn.BatchNorm1d(10)) for _ in range(10)])
        self.output = nn.Linear(10, 3)

    def forward(self, x):
        x = self.sequential(x)
        for module in self.repeat_linear:
            x = module(x)
        x = self.output(x)
        return x

In [20]:
model = Model()

In [21]:
model

Model(
  (sequential): Sequential(
    (0): Linear(in_features=12, out_features=10, bias=True)
    (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (repeat_linear): ModuleList(
    (0-9): 10 x Sequential(
      (0): Linear(in_features=10, out_features=10, bias=True)
      (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (output): Linear(in_features=10, out_features=3, bias=True)
)

In [25]:
input_ = torch.randn(10,12)
outupt = model(input_)

In [26]:
outupt

tensor([[ 0.5090, -1.3913, -0.4576],
        [-0.8579, -0.6577,  0.3451],
        [ 0.1382, -0.4318, -0.7107],
        [-0.0534, -0.2966,  0.2749],
        [-0.5043,  0.1860, -0.1980],
        [ 0.2435,  0.0631,  0.3992],
        [ 0.6860, -0.1278, -0.2720],
        [ 0.7831,  0.2836,  0.1239],
        [ 0.4415,  0.2105, -0.5201],
        [ 0.5165, -0.9239,  0.4835]], grad_fn=<AddmmBackward0>)